# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Liu  ->  J. Liu  |  ['J. Liu']
N. Martin  ->  N. Martin  |  ['N. Martin']
M. Fouesneau  ->  M. Fouesneau  |  ['M. Fouesneau']
M. Gross  ->  M. Gross  |  ['M. Gross']
Arxiv has 41 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2412.20348


extracting tarball to tmp_2412.20348... done.
Retrieving document from  https://arxiv.org/e-print/2412.20776


extracting tarball to tmp_2412.20776...

 done.


N. Martin  ->  N. Martin  |  ['N. Martin']
M. Fouesneau  ->  M. Fouesneau  |  ['M. Fouesneau']


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure Basti04_ages
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure Basti04_ages as tmp_2412.20776/./Basti04_ages.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


Found 50 bibliographic references in tmp_2412.20776/aa51517-24corr.bbl.
Retrieving document from  https://arxiv.org/e-print/2412.21034


extracting tarball to tmp_2412.21034...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.20776-b31b1b.svg)](https://arxiv.org/abs/2412.20776) | **The Pristine survey. XXVI. Chemical abundances of subgiant stars of the extremelymetal-poor stream C-19**  |
|| P. Bonifacio, et al. -- incl., <mark>N. Martin</mark>, <mark>M. Fouesneau</mark> |
|*Appeared on*| *2024-12-31*|
|*Comments*| *Astronomy & Astrophysics - A&A, In press*|
|**Abstract**|            Context: The C-19 stellar stream is the most metal-poor stream known to date. While its width and velocity dispersion indicate a dwarf galaxy origin, its metallicity spread and abundance patterns are more similar to those of globular clusters (GCs). If it is indeed of GC origin, its extremely low metallicity ([Fe/H]=-3.4, estimated from giant stars) implies that these stellar systems can form out of gas that is as extremely poor in metals as this. Previously, only giant stream stars were observed spectroscopically, although the majority of stream stars are unevolved stars. Aims: We pushed the spectroscopic observations to the subgiant branch stars ($G\approx 20$) in order to consolidate the chemical and dynamical properties of C-19. Methods: We used the high-efficiency spectrograph X-shooter fed by the ESO 8.2 m VLT telescope to observe 15 candidate subgiant C-19 members. The spectra were used to measure radial velocities and to determine chemical abundances using the \mygi\ code. Results; We developed a likelihood model that takes metallicity and radial velocities into account. We conclude that 12 stars are likely members of C-19, while 3 stars (S05, S12, and S13) are likely contaminants. When these 3 stars are excluded, our model implies a mean metallicity $\rm \langle [Fe/H]\rangle = -3.1\pm 0.1$, the mean radial velocity is $\langle v_r\rangle = -192\pm3$ kms$^{-1}$, and the velocity dispersion is $\sigma_{vr} = 5.9^{+3.6}_{-5.9}$ kms$^{-1}$. This all agrees within errors with previous studies. The A(Mg) of a sample of 15 C-19 members, including 6 giant stars, shows a standard deviation of 0.44 dex, and the mean uncertainty on Mg is 0.25 dex. Conclusions: Our preferred interpretation of the current data is that C-19 is a disrupted GC. We cannot completely rule out the possibility that the GC could have belonged to a dwarf galaxy that contained more metal-rich stars, however. This scenario would explain the radial velocity members at higher metallicity, as well as the width and velocity dispersion of the stream. In either case, a GC formed out of gas as poor in metals as these stars seems necessary to explain the existence of C-19. The possibility that no GC was associated with C-19 cannot be ruled out either.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.20348-b31b1b.svg)](https://arxiv.org/abs/2412.20348) | **Spatial and Temporal Distribution of Nanoflare Heating During Active Region Evolution**  |
|| B. Mondal, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2024-12-31*|
|*Comments*| *Accepted for publication in the Astrophysical Journal (ApJ)*|
|**Abstract**|            Nanoflares are believed to be key contributors to heating solar non-flaring active regions, though their individual detection remains challenging. This study uses a data-driven field-aligned hydrodynamic model to examine nanoflare properties throughout the lifecycle of AR12758. We simulate coronal loop emissions, where each loop is heated by random nanoflares depending on the loop parameters derived from photospheric magnetograms observed by SDO/HMI. Simulated X-ray flux and temperature can reproduce the temporal variations observed by Chandrayaan-2/XSM. Our findings show that high-frequency nanoflares contribute to cool emissions across the AR, while low- and intermediate- frequency primarily contribute to hot emissions. During the emerging phase, energy deposition is dominated by low-frequency events. Post-emergence, energy is deposited by both low- and intermediate-frequency nanoflares, while as the AR ages, the contribution from intermediate- and high-frequency nanoflares increases. The spatial distribution of heating frequencies across the AR reveals a clear pattern: the core of the active region spends most of its time in a low-frequency heating state, the periphery is dominated by high-frequency heating, and the region between the core and periphery experiences intermediate-frequency heating.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.21034-b31b1b.svg)](https://arxiv.org/abs/2412.21034) | **Novel SiC UV Instrumentation Development with Potential Applications for the Habitable Worlds Observatory**  |
|| P. Saxena, et al. -- incl., <mark>M. Gross</mark> |
|*Appeared on*| *2024-12-31*|
|*Comments*| *8 figures, Submitted to JATIS*|
|**Abstract**|            In this paper, we detail recent and current work that is being carried out to fabricate and advance novel SiC UV instrumentation that is aimed at enabling more sensitive measurements across numerous disciplines, with a short discussion of the promise such detectors may hold for the Habitable Worlds Observatory. We discuss SiC instrument development progress that is being carried out under multiple NASA grants, including several PICASSO and SBIR grants, as well as an ECI grant. Testing of pixel design, properties and layout as well as maturation of the integration scheme developed through these efforts provide key technology and engineering advancement for potential HWO detectors. Achieving desired noise characteristics, responsivity, and validating operation of SiC detectors using standard read out techniques offers a compelling platform for operation of denser and higher dimensionality SiC photodiode arrays of interest for use in potential HWO Coronagraph, Spectrograph, and High Resolution Imaging Instruments. We incorporate these SiC detector properties into a simulation of potential NUV exoplanet observations by HWO using SiC detectors and also discuss potential application to HWO.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2412.20776/./corner1.png', 'tmp_2412.20776/./Mg_Fe.png', 'tmp_2412.20776/./vrad.png']
copying  tmp_2412.20776/./corner1.png to _build/html/
copying  tmp_2412.20776/./Mg_Fe.png to _build/html/
copying  tmp_2412.20776/./vrad.png to _build/html/
exported in  _build/html/2412.20776.md
    + _build/html/tmp_2412.20776/./corner1.png
    + _build/html/tmp_2412.20776/./Mg_Fe.png
    + _build/html/tmp_2412.20776/./vrad.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\mygi}{MyGIsFOS}$
$\newcommand{\Teff}{\ensuremath{T_\mathrm{eff}}}$
$\newcommand{\gf}{\ensuremath{gf}}$
$\newcommand{\loggf}{\ensuremath{\log gf}}$
$\newcommand{\logg}{\ensuremath{\log g}}$
$\newcommand{\glog}{\ensuremath{\log g}}$
$\newcommand{\kms}{\rm km s ^{-1}}$
$\newcommand{\cobold}{CO^{5}BOLD}$
$\newcommand{\teff}{T\rm_{eff}}$</div>



<div id="title">

# The Pristine survey\ XXVI. Chemical abundances of subgiant stars of the extremelymetal-poor stream C-19$\thanks{Based on observations made at ESO with X-shooter, programme109.231F.001}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2412.20776-b31b1b.svg)](https://arxiv.org/abs/2412.20776)<mark>Appeared on: 2024-12-31</mark> -  _Astronomy & Astrophysics - A&A, In press_

</div>
<div id="authors">

P. Bonifacio, et al. -- incl., <mark>N. Martin</mark>, <mark>M. Fouesneau</mark>

</div>
<div id="abstract">

**Abstract:** The C-19 stellar stream is the most metal-poor stream known to date. While its width and velocity dispersion indicate a dwarf galaxy origin, its metallicity spread and abundance patterns are more similar to those of globular clusters (GCs). If it is indeed of GC origin,  its extremely low metallicity ( [ Fe/H ] =--3.4, estimated from giant stars)implies that these stellar systems can form out of gas that is as extremely poor in metals as this. Previously,only giant stream stars were observed spectroscopically, although the majorityof stream stars are unevolved stars. We pushed the spectroscopic observations to the subgiant branch stars( $G\approx 20$ ) in order to consolidate the chemical and dynamical propertiesof C-19. We used the high-efficiency spectrograph X-shooter fed bythe ESO 8.2 m VLT telescope to observe 15 candidate subgiant C-19 members.The spectra were used to measure radial velocities and to determinechemical abundances using the $\mygi$ code. We developed a likelihood model that takes metallicity and radial velocities into account. We conclude that12 stars are likely members of C-19, while 3 stars(S05, S12, and S13) are likely contaminants.When these 3 stars are excluded, our modelimplies a mean metallicity $\rm \langle[Fe/H]\rangle = -3.1\pm 0.1$ , the  mean radial velocity is $\langle v_r\rangle=-192\pm3$ $\kms$ ,  andthe velocity dispersion is $\sigma_{vr}=5.9^{+3.6}_{-5.9}$ $\kms$ . This all agrees within errors with previous studies. The A(Mg) of a sample of 15 C-19 members, including 6  giant stars, shows a  standard deviation of 0.44 dex, and the mean uncertainty on Mg is 0.25 dex. Our preferred interpretation of the current data is that C-19 is a disrupted GC.We cannot completely rule out  the possibility that the GC could have belonged to a dwarf galaxy that contained more metal-rich  stars, however. This  scenario would explain the radial velocity members at higher metallicity, as  well as the width and velocity dispersion of the stream. In either case, a GC formed out of gas as poor in metals as these stars seems necessary to explain the existence of C-19.  The possibility that no GC was associated with C-19 cannot be ruled out either.

</div>

<div id="div_fig1">

<img src="tmp_2412.20776/./corner1.png" alt="Fig2" width="100%"/>

**Figure 2. -** Probability distribution functions for the five parameters of the inference on the properties of the combined velocity and metallicity distribution of C-19 members. The five parameters are the fraction of contaminants, $\eta$, and the mean and dispersion of the velocity ($\langle v_r\rangle$ and $sigma_{vr}$) and metallicity ($\langle{\rm[Fe/H]}\rangle$ and $\sigma_{\rm[Fe/H]}$) parts of the model. The top panel of each column shows the one-dimensional marginalised probability distribution functions from which the favoured parameter models and related uncertainties are determined. The red contours and histograms show the same after the three likely contaminant stars are removed and the MW contamination is fixed to $\eta=0$. (*corner*)

</div>
<div id="div_fig2">

<img src="tmp_2412.20776/./Mg_Fe.png" alt="Fig5" width="100%"/>

**Figure 5. -** [Fe/H]--[Mg/Fe] diagram for our programme stars and the stars from
[Martin, Venn and Aguado (2022)]() and [Yuan, Martin and Ibata (2022)]() for which Mg abundances are available. (*mgfe*)

</div>
<div id="div_fig3">

<img src="tmp_2412.20776/./vrad.png" alt="Fig1" width="100%"/>

**Figure 1. -** Radial velocities of our program stars and the stars from [Martin, Venn and Aguado (2022)]() and
[Yuan, Martin and Ibata (2022)]() plotted as a function of declination.
The dashed lines indicate three
$\sigma$ shifts from the mean value (represented by the thick line). (*vrad*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2412.20776"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

292  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

2  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
